In [1]:
import pandas as pd
import Levenshtein
import itertools
from fuzzywuzzy import fuzz, process
import ast
from selenium import webdriver
import time
import webbrowser

In [2]:
recipe_df = pd.read_csv('../data/processed/ingredients_df.csv')

pd.set_option('max_columns', None)

recipe_df.head()

,recipe_name,recipe_url,ingredients_list,ingredient,variable,num_ingredients,category,main_category,tipo_plato,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free
0,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",patatas pequeñas rojas,0,12,ensaladas,principal,['Primeros platos y entrantes'],60.0,252.0,0,0,0,0,0
1,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",foie micuit,0,9,ensaladas,principal,['Primeros platos y entrantes'],25.0,386.0,0,0,0,0,0
2,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",bacalao desalado,0,16,ensaladas,principal,['Primeros platos y entrantes'],35.0,NaN,0,0,0,0,0
3,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",langostinos,0,12,ensaladas,principal,['Primeros platos y entrantes'],25.0,176.0,0,0,0,0,0
4,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",pechugas de pollo,0,16,ensaladas,principal,['Primeros platos y entrantes'],40.0,203.0,0,0,0,0,0


In [11]:
input_list = ['huevos', 'pechiga de pollo', 'lentejas', 'pimiento', 'arroz', 'leche', 'mantequilla',
              'harina', 'sal', 'azúcar', 'aceite', 'zanahorias', 'queso', 'bacon', 'lechuga', 'calabacín',
              'patatas', 'pollo']

In [12]:
def fuzzy_columns(column, input_string):
    ratio = fuzz.WRatio(column, input_string)
    return ratio

In [13]:
for ingred in input_list:
    recipe_df[ingred] = recipe_df.apply(lambda x: fuzzy_columns(x["ingredient"], ingred), axis=1)

In [14]:
recipe_df

,recipe_name,recipe_url,ingredients_list,ingredient,variable,num_ingredients,category,main_category,tipo_plato,time_preparation(min),calories(kcal),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,huevos,pechiga de pollo,lentejas,pimiento,arroz,leche,mantequilla,harina,sal,azúcar,aceite,zanahorias,queso,bacon,lechuga,calabacín,patatas,pollo
0,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevos', 'judías v...",patatas pequeñas rojas,0,12,ensaladas,principal,['Primeros platos y entrantes'],60.0,252.0,0,0,0,0,0,45,32,34,22,36,36,40,30,30,36,30,36,72,18,39,34,90,18
1,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",foie micuit,0,9,ensaladas,principal,['Primeros platos y entrantes'],25.0,386.0,0,0,0,0,0,15,28,21,42,12,25,27,15,0,18,45,19,18,18,39,11,13,18
2,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",bacalao desalado,0,16,ensaladas,principal,['Primeros platos y entrantes'],35.0,NaN,0,0,0,0,0,30,44,34,22,36,19,30,30,90,54,30,36,36,54,26,45,39,36
3,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",langostinos,0,12,ensaladas,principal,['Primeros platos y entrantes'],25.0,176.0,0,0,0,0,0,30,22,42,32,36,18,36,35,30,18,30,48,36,38,26,32,33,25
4,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",pechugas de pollo,0,16,ensaladas,principal,['Primeros platos y entrantes'],40.0,203.0,0,0,0,0,0,45,91,34,24,34,54,32,30,30,18,30,26,36,18,77,24,26,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57309,Pastel de tres colores y tres sabores,https://www.lecturas.com/recetas/pastel-tres-c...,"['mantequilla', 'azúcar', 'huevo', 'aroma de v...",azúcar,30,33,postres y dulces,postre,['Postres'],240.0,NaN,0,0,0,0,0,0,19,18,0,40,20,25,36,30,100,36,40,0,40,33,36,33,0
57310,Tarta de cubos de chocolate con toffee,https://www.lecturas.com/recetas/tarta-cubos-c...,"['mantequilla', 'azúcar', 'huevo', 'harina', '...",chocolate negro fondant,31,33,postres y dulces,postre,['Postres'],360.0,NaN,0,0,0,0,0,30,36,45,34,36,36,31,34,30,36,45,27,36,36,27,45,27,36
57311,Pastel de tres colores y tres sabores,https://www.lecturas.com/recetas/pastel-tres-c...,"['mantequilla', 'azúcar', 'huevo', 'aroma de v...",chocolate negro fondant,31,33,postres y dulces,postre,['Postres'],240.0,NaN,0,0,0,0,0,30,36,45,34,36,36,31,34,30,36,45,27,36,36,27,45,27,36
57312,Tarta de cubos de chocolate con toffee,https://www.lecturas.com/recetas/tarta-cubos-c...,"['mantequilla', 'azúcar', 'huevo', 'harina', '...",cacao en polvo puro,32,33,postres y dulces,postre,['Postres'],360.0,NaN,0,0,0,0,0,30,51,22,22,36,36,24,30,30,54,45,28,36,54,26,45,26,72


In [15]:
input_list

['huevos',
 'pechiga de pollo',
 'lentejas',
 'pimiento',
 'arroz',
 'leche',
 'mantequilla',
 'harina',
 'sal',
 'azúcar',
 'aceite',
 'zanahorias',
 'queso',
 'bacon',
 'lechuga',
 'calabacín',
 'patatas',
 'pollo']

In [46]:
filter_huevos = recipe_df['huevos'] > 80
recipe_df[filter_huevos]["ingredient"].value_counts()

huevo                                          1373
huevos                                          723
yema de huevo                                   329
clara de huevo                                  194
yemas de huevo                                  101
claras de huevo                                  76
huevos de codorniz                               35
huevos grandes                                   10
huevos medianos                                   6
huevos duros                                      2
huevos pequeños                                   1
fideos chinos de huevo                            1
huevos de chocolate                               1
huevos cocidos                                    1
huevos cocido                                     1
fideos de huevo                                   1
aceite de oliva suave el peso de los huevos       1
harina integral el peso de los huevos             1
azúcar moreno el peso de los huevos               1
Name: ingred

In [33]:
filter_pechiga_de_pollo = recipe_df['pechiga de pollo'] > 86
recipe_df[filter_pechiga_de_pollo]["ingredient"].value_counts()

pechugas de pollo    86
pechuga de pollo     14
pollo                 4
Name: ingredient, dtype: int64

In [43]:
filter_lentejas = recipe_df['lentejas'] > 86
recipe_df[filter_lentejas]["ingredient"].value_counts()

lentejas                      23
lentejas cocidas              11
lentejas pardinas              8
lentejas pardinas cocidas      8
lentejas rojas                 2
lentejas rojas lavados         1
hummus de lentejas rojas       1
lentejas del tipo pardinas     1
paté de lentejas               1
Name: ingredient, dtype: int64

In [44]:
filter_pimiento = recipe_df['pimiento'] > 86
recipe_df[filter_pimiento]["ingredient"].value_counts()

pimienta                                 1484
pimiento rojo                             318
pimiento verde                            278
pimientos rojos                            89
pimientos verdes                           60
                                         ... 
pimientos verdes alargados                  1
pimientos del piquillo envasados            1
pimientos choriceros de la pulpa            1
pimientos verdes italianos en daditos       1
pimiento rojo 1 kilo                        1
Name: ingredient, Length: 83, dtype: int64